# NanoChat Inference Demo (Google Colab)

This notebook shows how to load the NanoChat speedrun checkpoint (`royam0820/nanochat-speedrun-001`) and generate responses programmatically in Google Colab.

**Note:** Make sure to enable GPU in Colab: Runtime → Change runtime type → GPU


In [ ]:
# Install dependencies
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install datasets fastapi uvicorn wandb huggingface-hub tokenizers tiktoken regex psutil


In [ ]:
# Clone nanochat repository and install
import os
import sys

# Install Rust (needed for building the rustbpe extension)
os.system("curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y")
os.environ["PATH"] = os.path.expanduser("~/.cargo/bin") + ":" + os.environ.get("PATH", "")

# Clone the repo (or use your fork)
repo_url = "https://github.com/karpathy/nanochat.git"  # Change this if using a fork
repo_dir = "nanochat"

if not os.path.exists(repo_dir):
    os.system(f"git clone {repo_url}")
    
# Add to Python path
sys.path.insert(0, repo_dir)

# Install the package (this will build the Rust extension)
# Note: This may take a few minutes to compile the Rust extension
os.system(f"cd {repo_dir} && pip install -e .")


In [ ]:
# Download model weights and tokenizer from Hugging Face using snapshot_download
from huggingface_hub import snapshot_download
import os
import shutil

# Model configuration
model_id = "royam0820/nanochat-speedrun-001"
print(f"Downloading {model_id}...")

# Use snapshot_download to get all files from the model repo
# This downloads everything in one go (model.pt, meta_*.json, tokenizer.pkl if available)
snapshot_path = snapshot_download(
    repo_id=model_id,
    repo_type="model",
    local_dir="out",
    local_dir_use_symlinks=False,  # Use actual files, not symlinks
)

print(f"✅ Model downloaded to: {snapshot_path}")

# Set up the checkpoint directory structure
cache_dir = os.path.expanduser("~/.cache/nanochat")
checkpoint_dir = os.path.join(cache_dir, "chatsft_checkpoints", "d20")
os.makedirs(checkpoint_dir, exist_ok=True)

# Copy the model files to the expected location
if os.path.exists("out/model.pt"):
    shutil.copy("out/model.pt", os.path.join(checkpoint_dir, "model_000700.pt"))
    print(f"✅ Model weights copied to {checkpoint_dir}")
else:
    raise FileNotFoundError("model.pt not found in downloaded snapshot")

if os.path.exists("out/meta_000700.json"):
    shutil.copy("out/meta_000700.json", os.path.join(checkpoint_dir, "meta_000700.json"))
    print(f"✅ Metadata copied to {checkpoint_dir}")
else:
    print("⚠️  meta_000700.json not found, but continuing...")

print(f"\nModel files set up in {checkpoint_dir}")


In [ ]:
# Download or recreate tokenizer
# First check if tokenizer was downloaded via snapshot_download, then try hf_hub_download
from huggingface_hub import hf_hub_download
import os
import shutil

# Set up the tokenizer directory structure
cache_dir = os.path.expanduser("~/.cache/nanochat")
tokenizer_dir = os.path.join(cache_dir, "tokenizer")
os.makedirs(tokenizer_dir, exist_ok=True)

model_id = "royam0820/nanochat-speedrun-001"
tokenizer_path = os.path.join(tokenizer_dir, "tokenizer.pkl")

# Check if tokenizer was already downloaded via snapshot_download
if os.path.exists("out/tokenizer.pkl"):
    print("✅ Tokenizer found in snapshot_download output")
    shutil.copy("out/tokenizer.pkl", tokenizer_path)
    print(f"✅ Tokenizer copied to {tokenizer_path}")
else:
    print(f"Tokenizer not in snapshot, attempting to download from {model_id}...")
    try:
        # Try to download tokenizer.pkl from the repo using hf_hub_download
        downloaded_path = hf_hub_download(
            repo_id=model_id,
            filename="tokenizer.pkl",
            repo_type="model",
        )
        # Copy to the expected location
        shutil.copy(downloaded_path, tokenizer_path)
        print(f"✅ Tokenizer downloaded to {tokenizer_path}")
    except Exception as e:
        print(f"⚠️  Tokenizer not found in Hugging Face repo: {e}")
        print("\n" + "="*60)
        print("OPTION 1: Upload your local tokenizer to Hugging Face")
        print("="*60)
        print("From your local environment where inference works, run:")
        print("  python -m scripts.upload_tokenizer")
        print("\nOr manually:")
        print("  from huggingface_hub import HfApi")
        print("  api = HfApi()")
        print(f"  api.upload_file(")
        print(f"      path_or_fileobj='~/.cache/nanochat/tokenizer/tokenizer.pkl',")
        print(f"      path_in_repo='tokenizer.pkl',")
        print(f"      repo_id='{model_id}',")
        print(f"      repo_type='model'")
        print(f"  )")
        print("\n" + "="*60)
        print("OPTION 2: Recreate tokenizer in Colab (slower, requires data)")
        print("="*60)
        print("This will train a new tokenizer with the same parameters.")
        print("Run the next cell to recreate it.")
        print("\n" + "="*60)
        raise FileNotFoundError(
            f"Tokenizer not found. Please upload tokenizer.pkl to {model_id} "
            "or use the next cell to recreate it."
        )


## Alternative: Recreate Tokenizer (if not in HF repo)

If the tokenizer is not available in the Hugging Face repo, you can recreate it here.
**Note:** This requires downloading training data and may take 10-20 minutes.


In [ ]:
# Recreate tokenizer (only run this if tokenizer download failed above)
# This trains a new tokenizer with the same parameters as the original model
# Requires: vocab_size=65536, max_chars=2B (same as speedrun.sh)

import os
import sys
from nanochat.tokenizer import RustBPETokenizer
from nanochat.common import get_base_dir
from nanochat.dataset import parquets_iter_batched

# Check if tokenizer already exists
cache_dir = os.path.expanduser("~/.cache/nanochat")
tokenizer_dir = os.path.join(cache_dir, "tokenizer")
tokenizer_path = os.path.join(tokenizer_dir, "tokenizer.pkl")

if os.path.exists(tokenizer_path):
    print(f"✅ Tokenizer already exists at {tokenizer_path}")
    print("Skipping recreation.")
else:
    # Parameters matching the original training (from meta_000700.json: vocab_size=65536)
    VOCAB_SIZE = 65536
    MAX_CHARS = 2_000_000_000  # 2B characters (same as speedrun.sh)

    print("="*60)
    print("RECREATING TOKENIZER")
    print("="*60)
    print(f"Vocab size: {VOCAB_SIZE}")
    print(f"Max chars: {MAX_CHARS:,}")
    print("This will download training data and train the tokenizer...")
    print("This may take 10-20 minutes.")
    print("="*60)

    # Download training data (8 shards for ~2B chars)
    print("\n📥 Downloading training data...")
    os.system("python -m nanochat.dataset -n 8")

    # Text iterator
    def text_iterator():
        nchars = 0
        for batch in parquets_iter_batched(split="train"):
            for doc in batch:
                doc_text = doc[:10_000]  # doc_cap
                nchars += len(doc_text)
                yield doc_text
                if nchars > MAX_CHARS:
                    return

    print("\n🔨 Training tokenizer...")
    text_iter = text_iterator()
    tokenizer = RustBPETokenizer.train_from_iterator(text_iter, VOCAB_SIZE)

    # Save the tokenizer
    os.makedirs(tokenizer_dir, exist_ok=True)
    tokenizer.save(tokenizer_dir)

    print(f"\n✅ Tokenizer recreated and saved to {tokenizer_path}")
    print("\n💡 Tip: Upload this to Hugging Face for faster future use:")
    print(f"   python -m scripts.upload_tokenizer {tokenizer_path}")


In [ ]:
# Load the model
import torch
from nanochat.common import compute_init, autodetect_device_type
from nanochat.checkpoint_manager import load_model
from nanochat.engine import Engine

device_type = autodetect_device_type()
_, _, _, _, device = compute_init(device_type)

model, tokenizer, meta = load_model(
    source="sft",
    device=device,
    phase="eval",
    model_tag="d20",
    step=700,
)

print(f"Loaded NanoChat model on {device}")


In [ ]:
# Set up the inference engine
from contextlib import nullcontext

bos = tokenizer.get_bos_token_id()
user_start = tokenizer.encode_special("<|user_start|>")
user_end = tokenizer.encode_special("<|user_end|>")
assistant_start = tokenizer.encode_special("<|assistant_start|>")
assistant_end = tokenizer.encode_special("<|assistant_end|>")

engine = Engine(model, tokenizer)

if device_type == "cuda":
    autocast_ctx = torch.amp.autocast(device_type=device_type, dtype=torch.bfloat16)
else:
    autocast_ctx = nullcontext()


def generate_response(prompt, temperature=0.6, top_k=50, max_tokens=256):
    conversation_tokens = [bos]
    conversation_tokens.append(user_start)
    conversation_tokens.extend(tokenizer.encode(prompt))
    conversation_tokens.append(user_end)
    conversation_tokens.append(assistant_start)

    response_tokens = []
    with autocast_ctx:
        for token_column, token_masks in engine.generate(
            conversation_tokens,
            num_samples=1,
            max_tokens=max_tokens,
            temperature=temperature,
            top_k=top_k,
        ):
            token = token_column[0]
            response_tokens.append(token)
            if token == assistant_end:
                break

    if response_tokens and response_tokens[-1] != assistant_end:
        response_tokens.append(assistant_end)

    # Decode without the final assistant_end token
    text = tokenizer.decode(response_tokens[:-1])
    return text.strip()


In [ ]:
# Single prompt example
sample_prompt = "Hello there! What can you do?"
response = generate_response(sample_prompt)
print(f"Prompt: {sample_prompt}\n")
print("NanoChat:")
print(response)


In [ ]:
# Batch prompts example
def generate_batch_responses(prompts, temperature=0.6, top_k=50, max_tokens=256):
    results = []
    for prompt in prompts:
        text = generate_response(prompt, temperature=temperature, top_k=top_k, max_tokens=max_tokens)
        results.append({"prompt": prompt, "response": text})
    return results

batch_prompts = [
    "Explain why the sky appears blue.",
    "Give me a haiku about autumn.",
    "What is 7 multiplied by 13?",
]

for item in generate_batch_responses(batch_prompts):
    print(f"Prompt: {item['prompt']}")
    print(f"Response: {item['response']}\n")
